<a href="https://colab.research.google.com/github/MoritzLaurer/zeroshot-classifier/blob/main/4_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training and Evaluation
This script takes the prepared data from the other scripts and trains and evaluates a model.

### Install and setup
Note that this notebook was written and tested in Google Colab to increase reproducibility & accessibility. The final run, however, was executed on a university HPC system on an A100 GPU for convenience. With a Google Colab Pro subscription, you should also be able to run the notebook. It will take several hours.  

In [ ]:
# for versioning of experiments with W&B
DATE = 20240314

SEED_GLOBAL = 42

import os

USING_COLAB = False
USING_NOTEBOOK = False

"""if 'COLAB_GPU' in os.environ or 'COLAB_JUPYTER_TRANSPORT' in os.environ or 'COLAB_BACKEND_VERSION' in os.environ:
    USING_COLAB = True
else:
    USING_COLAB = False"""

if USING_COLAB or USING_NOTEBOOK:
    # comment this away this if you are not using colab
    """!pip install transformers[sentencepiece]~=4.37.0 -q
    !pip install datasets~=2.17.0 -q
    !pip install accelerate~=0.23.0 -q
    !pip install wandb~=0.16.3 -q
    !pip install mdutils~=1.6.0 -q
    !pip install scikit-learn~=1.2.0 -q"""

In [ ]:
## load packages
import pandas as pd
import numpy as np
import os
from datasets import load_dataset, concatenate_datasets
import re
import time
import random
import tqdm
import torch
from torch.utils.data import DataLoader

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import ClassLabel
from datasets import load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets, list_metrics

from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report

import gc
from accelerate.utils import release_memory

import wandb
import json
from datetime import datetime
import argparse

from mdutils import MdUtils

# suppress unnecessary tokenizer warning https://github.com/huggingface/transformers/issues/14285
transformers.logging.set_verbosity_error()

np.random.seed(SEED_GLOBAL)
torch.manual_seed(SEED_GLOBAL)
random.seed(SEED_GLOBAL)

In [ ]:
if USING_COLAB or USING_NOTEBOOK:
    # info on the GPU you are using
    #!nvidia-smi
    # info on available ram
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('\n\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

In [ ]:
if USING_COLAB:
    ## connect to google drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

    #set wd
    print(os.getcwd())
    os.chdir("/content/drive/My Drive/PhD/zero-shot-models")

print(os.getcwd())

# local config.py file with tokens
import config

In [ ]:
## set main arguments
# setup this way to also enable submitting the script via HPC systems
parser = argparse.ArgumentParser(description='Pass arguments via terminal')

# make sure that parsing of boolian args is done correctly
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

# main args
parser.add_argument('-ds_heldout', '--dataset_name_heldout', type=str,
                    help='names for datasets not to include in training. For heldout testing')
parser.add_argument('-train', '--do_train', type=str2bool, default=True,
                    help='Do training of flag set. Otherwise only evaluation')
parser.add_argument('-upload', '--upload_to_hub',type=str2bool, default=False,
                    help='Upload model to HF hub if flag is set')


if USING_COLAB or USING_NOTEBOOK:
    args = parser.parse_args([
        "--dataset_name_heldout", "none",  #"all_except_nli",
        # comment following arguments away to set them to False
        "--do_train", "False",
        "--upload_to_hub", "False"
    ])
    print("Manually defined arguments:\n", args)
else:
    args = parser.parse_args()
    print("Arguments passed via the terminal:\n", args)


### Load data

In [ ]:
## load and merge training data
n_downsample = 500_000
mixtral_datasets = ['mixtral_written_texts_for_tasks_v4', "mixtral_refinedweb_nli"]  #['mixtral_refinedweb_categories', 'mixtral_refinedweb_characteristics', 'mixtral_refinedweb_nli', 'mixtral_written_texts_for_tasks', 'mixtral_written_texts_for_tasks_v2]
print("Synthetic datasets used: ", mixtral_datasets)

dataset_synthetic_lst = []
for dataset_name in mixtral_datasets:
    dataset_synthetic = load_dataset("MoritzLaurer/synthetic_zeroshot_mixtral_v0.1", dataset_name, token=config.HF_ACCESS_TOKEN)["train"]
    dataset_synthetic = dataset_synthetic.select_columns(["hypothesis", "text", "labels"])
    dataset_synthetic = dataset_synthetic.add_column("task_name", [dataset_name] * len(dataset_synthetic))
    # downsample for testing
    if len(dataset_synthetic) > n_downsample:
        dataset_synthetic = dataset_synthetic.shuffle(seed=42)
        dataset_synthetic = dataset_synthetic.select(range((min(n_downsample, len(dataset_synthetic)))))
    dataset_synthetic_lst.append(dataset_synthetic)

dataset_synth = concatenate_datasets(dataset_synthetic_lst)
print("Synthetic dataset length: ", len(dataset_synth))

In [ ]:
# introduce variation to the hypotheses. In line with HF zeroshot pipeline template
# https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.ZeroShotClassificationPipeline.__call__
# Calculate X% of your dataset size
dataset_size = len(dataset_synth)  # Assuming you're working with the loaded dataset
num_to_select = int(dataset_size * 0.5)

# Randomly select indices
rng = np.random.default_rng(42)  # Initialize the RNG with a seed for reproducibility
selected_indices = rng.choice(dataset_size, num_to_select, replace=False)

prefix_lst = ["This example", "This example text"]

# Define a function to replace the string only for selected indices
def replace_text(examples, indices):
    modified_hypotheses = []
    for idx, hypothesis in zip(indices, examples['hypothesis']):
        if idx in selected_indices and hypothesis.startswith('The text'):
            rng = np.random.default_rng(idx)
            chosen_prefix = rng.choice(prefix_lst)
            modified_hypotheses.append(hypothesis.replace('The text', chosen_prefix, 1))
        else:
            modified_hypotheses.append(hypothesis)
    examples['hypothesis'] = modified_hypotheses
    return examples

# Apply the function to the dataset
dataset_synth = dataset_synth.map(replace_text, with_indices=True, batched=True)

# Output a sample to verify changes
for i in range(10):
    print(dataset_synth[i]['hypothesis'])


In [ ]:
# load human NLI data and merge with synthetic data
dataset_mnli_fever = load_dataset("MoritzLaurer/mnli_fever_clean", token=config.HF_ACCESS_TOKEN)["train"]
# only maintain selected columns
dataset_mnli_fever = dataset_mnli_fever.select_columns(["hypothesis", "text", "labels", "dataset"])
# rename column
dataset_mnli_fever = dataset_mnli_fever.rename_column("dataset", "task_name")

# concatenate
dataset_train = concatenate_datasets([dataset_mnli_fever] + [dataset_synth])
# shuffle dataset
dataset_train = dataset_train.shuffle(seed=SEED_GLOBAL)

dataset_train

In [ ]:
# load from hub
#dataset_train = load_dataset("MoritzLaurer/dataset_train_nli", token=config.HF_ACCESS_TOKEN)["train"]
dataset_test_concat_nli = load_dataset("MoritzLaurer/dataset_test_concat_nli", token=config.HF_ACCESS_TOKEN)["train"]
#dataset_test_disaggregated = load_dataset("MoritzLaurer/dataset_test_disaggregated_nli", token=config.HF_ACCESS_TOKEN)
dataset_test_disaggregated = load_dataset("MoritzLaurer/zeroshot_test_downsampled", token=config.HF_ACCESS_TOKEN)


print(dataset_test_concat_nli)
print(dataset_test_disaggregated)

In [ ]:
# manually written task names for validating that code doesn't miss anything
task_names_manual = [
    'wellformedquery', 'financialphrasebank', 'rottentomatoes', 'amazonpolarity',
    'imdb', 'appreviews', 'yelpreviews', 'wikitoxic_toxicaggregated',
    'wikitoxic_obscene', 'wikitoxic_threat', 'wikitoxic_insult',
    'wikitoxic_identityhate', 'hateoffensive', 'hatexplain',
    'trueteacher', 'spam', 'massive', 'banking77', 'emotiondair',
    'emocontext', 'empathetic', 'agnews', 'yahootopics',
    'biasframes_offensive', 'biasframes_sex', 'biasframes_intent',
    # added for v1.1
    "manifesto", "capsotu",
]

In [ ]:
# select specific training subset
if args.dataset_name_heldout == "all_except_nli":
    dataset_name_only_nli = ["mnli", "anli", "fevernli", "wanli", "lingnli"]
    dataset_names_lst = set(dataset_train['task_name'])
    dataset_name_heldout = [dataset_name for dataset_name in dataset_names_lst if dataset_name not in dataset_name_only_nli]
else:
    dataset_name_heldout = [args.dataset_name_heldout]

print(dataset_train.to_pandas().task_name.value_counts())

dataset_train_filt = dataset_train.filter(lambda example: example['task_name'] not in dataset_name_heldout)

print(dataset_train_filt.to_pandas().task_name.value_counts())


In [ ]:
# downsampling for faster testing
"""dataset_train_filt = dataset_train_filt.select(range(1000))
dataset_test_concat_nli = dataset_test_concat_nli.select(range(1000))
for dataset in dataset_test_disaggregated:
    dataset_test_disaggregated[dataset] = dataset_test_disaggregated[dataset].select(range(20))"""

### Tokenize, train eval

In [ ]:
### Load model and tokenizer

if args.do_train:
    model_name = "microsoft/deberta-v3-large"  # "facebook/bart-large-mnli", "microsoft/deberta-v3-xsmall"  #"microsoft/deberta-v3-large"  #"microsoft/deberta-v3-base" # microsoft/xtremedistil-l6-h256-uncased
else:
    # can only comprehensively test binary NLI models, because NLI test datasets are binarized
    model_name = "MoritzLaurer/deberta-v3-large-zeroshot-v2.0-2024-03-20-21-16"  #"facebook/bart-large-mnli", "MoritzLaurer/deberta-v3-base-mnli-fever-anli-ling-wanli-binary"   #"sileod/deberta-v3-base-tasksource-nli"  #"MoritzLaurer/DeBERTa-v3-base-mnli-fever-docnli-ling-2c"

max_length = 512

## load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# label2id mapping
if args.do_train:
    label2id = {"entailment": 0, "not_entailment": 1}  #{"entailment": 0, "neutral": 1, "contradiction": 2}
    id2label = {0: "entailment", 1: "not_entailment"}  #{0: "entailment", 1: "neutral", 2: "contradiction"}

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, model_max_length=max_length)  # model_max_length=512
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, label2id=label2id, id2label=id2label
    ).to(device)

    label_text_unique = list(label2id.keys())
    print(label_text_unique)

else:
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, model_max_length=max_length)  # model_max_length=512
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name
    ).to(device)

    label_text_unique = list(model.config.id2label.values())
    print(label_text_unique)



### Configure logging with W&B

In [ ]:
## logging with wandb

# logging
wandb.login(key=config.WANDB_ACCESS_TOKEN)

project_name = f"nli-zeroshot-{DATE}"

now = datetime.now().strftime("%Y-%m-%d-%H-%M")
run_name = f"{model_name.split('/')[-1]}-zeroshot-{now}"

#wandb.init(project=project_name, name=run_name)
# if updating config here, HF trainer does not seem to log info to config anymore
#wandb.config.update({"dataset_name_heldout": dataset_name_heldout}, allow_val_change=True)

# https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/callback#transformers.integrations.WandbCallback
os.environ["WANDB_PROJECT"] = project_name  # log to your project
#%env WANDB_PROJECT=amazon_sentiment_analysis
os.environ["WANDB_LOG_MODEL"] = "false"  # Can be "end", "checkpoint" or "false". If set to "end", the model will be uploaded at the end of training. If set to "checkpoint", the checkpoint will be uploaded every args.save_steps . If set to "false", the model will not be uploaded. Use along with load_best_model_at_end() to upload best model.
os.environ["WANDB_WATCH"] = "false"   # Can be "gradients", "all", "parameters", or "false". Set to "all" to log gradients and parameters.

# custom init https://docs.wandb.ai/guides/integrations/huggingface#customize-wandbinit
# config and name initialized via HF trainer
#wandb.init(project="my_project", config=experiment['config'], name=experiment['name']):

#### Tokenize

In [ ]:
# Dynamic padding, HF course: https://huggingface.co/course/chapter3/2?fw=pt

# without padding="max_length" & max_length=512, it should do dynamic padding.
def tokenize_func(examples):
    return tokenizer(examples["text"], examples["hypothesis"], truncation=True)  # max_length=512,  padding=True

# training on:
encoded_dataset_train = dataset_train_filt.map(tokenize_func, batched=True)
print(len(encoded_dataset_train))
# testing during training loop on aggregated testset:
encoded_dataset_test = dataset_test_concat_nli.map(tokenize_func, batched=True)
print(len(encoded_dataset_test))
# testing on individual datasets:
encoded_dataset_test_disaggregated = dataset_test_disaggregated.map(tokenize_func, batched=True)

# remove columns the library does not expect
encoded_dataset_train = encoded_dataset_train.remove_columns(["hypothesis", "text"])
encoded_dataset_test = encoded_dataset_test.remove_columns(["hypothesis", "text"])


#### Training

In [ ]:
# release memory: https://huggingface.co/blog/optimize-llm

def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()


In [ ]:
# function for computing metrics for normally formatted classification tasks
# here, this is used for the standard NLI datasets like MNLI, ANLI etc
def compute_metrics_standard(eval_pred, label_text_alphabetical=None):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)  # argmax on each row (axis=1) in the tensor

    # metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)

    metrics = {'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            #'label_gold_raw': labels,
            #'label_predicted_raw': preds_max
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} )  # print metrics but without label lists
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics


# function to compute metrics for classification tasks that have been reformatted into the NLI format
# here, this is used for the non-NLI classification tasks (which were converted to NLI format)
def compute_metrics_nli_binary(eval_pred, label_text_alphabetical=None):
    print("Calculating metrics for label_text_alphabetical: ", label_text_alphabetical)
    
    # special handling for BART encoder-decoder model
    if "bart" in model_name:
        labels = eval_pred.label_ids
        predictions = eval_pred.predictions[0]
    else:
        predictions, labels = eval_pred
    
    # split in chunks with predictions for each hypothesis for one unique premise
    def chunks(lst, n):  # Yield successive n-sized chunks from lst. https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # for each chunk/premise, select the most likely hypothesis, either via raw logits, or softmax
    prediction_chunks_lst = list(chunks(predictions, len(set(label_text_alphabetical)) ))  # len(LABEL_TEXT_ALPHABETICAL)
    hypo_position_highest_prob = []
    for i, chunk in enumerate(prediction_chunks_lst):
        # only accesses the first column of the array, i.e. the entailment prediction logit of all hypos and takes the highest one
        if "bart" not in model_name.lower():  # my 0-shot models have label sequence ['entailment', 'not_entailment']
            hypo_position_highest_prob.append(np.argmax(chunk[:, 0]))
        else:  # bart has label sequence ['contradiction', 'neutral', 'entailment']
            hypo_position_highest_prob.append(np.argmax(chunk[:, 2]))

    label_chunks_lst = list(chunks(labels, len(set(label_text_alphabetical)) ))
    label_position_gold = []
    for chunk in label_chunks_lst:
        label_position_gold.append(np.argmin(chunk))  # argmin to detect the position of the 0 among the 1s

    # for inspection
    print("Highest probability prediction per premise: ", hypo_position_highest_prob[:10])
    print("Correct label per premise: ", label_position_gold[:10])

    ## metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(label_position_gold, hypo_position_highest_prob, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(label_position_gold, hypo_position_highest_prob, average='micro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    acc_balanced = balanced_accuracy_score(label_position_gold, hypo_position_highest_prob)
    acc_not_balanced = accuracy_score(label_position_gold, hypo_position_highest_prob)
    metrics = {'f1_macro': f1_macro,
               'f1_micro': f1_micro,
               'accuracy_balanced': acc_balanced,
               'accuracy': acc_not_balanced,
               'precision_macro': precision_macro,
               'recall_macro': recall_macro,
               'precision_micro': precision_micro,
               'recall_micro': recall_micro,
               #'label_gold_raw': label_position_gold,
               #'label_predicted_raw': hypo_position_highest_prob
               }
    print("Aggregate metrics: ", {
        key: metrics[key] for key in metrics
        if key not in ["label_gold_raw", "label_predicted_raw"]
    })
    print("Detailed metrics: ", classification_report(
        label_position_gold,
        hypo_position_highest_prob,
        labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical,
        sample_weight=None, digits=2, output_dict=True,
        zero_division='warn'),
    "\n")

    return metrics


In [ ]:
# special logits handler for trainer given BART logits memory leak issue
# https://discuss.huggingface.co/t/cuda-out-of-memory-when-using-trainer-with-compute-metrics/2941/13?u=moritzlaurer

def preprocess_logits_for_metrics_bart(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    #pred_ids = torch.argmax(logits[0], dim=-1)
    logits = logits[0]
    return logits, labels

In [ ]:
training_directory = f'./results/{model_name.split("/")[-1]}-zeroshot-{now}'

fp16_train, fp16_eval = (True, True) if torch.cuda.is_available() else (False, False)
if "mdeberta" in model_name.lower(): fp16_train, fp16_eval = (False, False)  # mDeBERTa does not support FP16 yet
if "large" in model_name.lower(): fp16_train, fp16_eval = (False, True)  # batch size below 16 for large, so FP16 does not add value

# https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments
eval_batch = 32 if "large" in model_name else 64*2
print("eval_batch size: ", eval_batch)
per_device_train_batch_size = 4 if "large" in model_name else 16
gradient_accumulation_steps = 8 if "large" in model_name else 2

#if USING_COLAB:
    #per_device_train_batch_size = int(per_device_train_batch_size / 4)
    #gradient_accumulation_steps = int(gradient_accumulation_steps * 4)
    #eval_batch = int(eval_batch / 32) if "large" in model_name else int(eval_batch / 8)

hub_model_id = f'MoritzLaurer/{model_name.split("/")[-1]}-zeroshot-v2.0-{now}'

train_args = TrainingArguments(
    output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    #deepspeed="ds_config_zero3.json",  # if using deepspeed
    lr_scheduler_type= "linear",
    group_by_length=False,  # can increase speed with dynamic padding, by grouping similar length texts https://huggingface.co/transformers/main_classes/trainer.html
    learning_rate=9e-6 if "large" in model_name else 2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=eval_batch,
    gradient_accumulation_steps=gradient_accumulation_steps,  # (!adapt/halve batch size accordingly). accumulates gradients over X steps, only then backward/update. decreases memory usage, but also slightly speed
    #eval_accumulation_steps=2,
    num_train_epochs=2,
    #max_steps=400,
    #warmup_steps=0,  # 1000,
    warmup_ratio=0.06,  #0.1, 0.06
    weight_decay=0.01,  #0.1,
    fp16=fp16_train,   # ! only makes sense at batch-size > 8. loads two copies of model weights, which creates overhead. https://huggingface.co/transformers/performance.html?#fp16
    fp16_full_eval=fp16_eval,
    evaluation_strategy="epoch",
    seed=SEED_GLOBAL,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy",
    #eval_steps=300,  # evaluate after n steps if evaluation_strategy!='steps'. defaults to logging_steps
    save_strategy="no",  # options: "no"/"steps"/"epoch"
    #save_steps=1_000_000,  # Number of updates steps before two checkpoint saves.
    save_total_limit=1,  # If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir
    #logging_strategy="epoch",
    report_to="all",  # "all"
    run_name=run_name,
    push_to_hub=args.upload_to_hub,  # does not seem to work if save_strategy="no"
    hub_model_id=hub_model_id,
    hub_token=config.HF_ACCESS_TOKEN,
    hub_strategy="end",
    hub_private_repo=True,
)


In [ ]:
hub_model_id

In [ ]:
trainer = Trainer(
    model=model,
    #model_init=model_init,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=encoded_dataset_train,  #.shard(index=1, num_shards=200),  # https://huggingface.co/docs/datasets/processing.html#sharding-the-dataset-shard
    eval_dataset=encoded_dataset_test,  #.shard(index=1, num_shards=20),
    compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=label_text_unique),  #compute_metrics,
    #data_collator=data_collator,  # for weighted sampling per dataset; for dynamic padding probably not necessary because done by default  https://huggingface.co/course/chapter3/3?fw=pt
    preprocess_logits_for_metrics=preprocess_logits_for_metrics_bart if "bart" in model_name else None,
)

if device == "cuda":
    # free memory
    flush()
    release_memory(model)
    #del (model, trainer)


In [ ]:
# train
if args.do_train:
    trainer.train()


#### Evaluation

In [ ]:
# could load specific model for evaluation here
#model = AutoModelForSequenceClassification.from_pretrained('./results/nli-few-shot/all-nli-3c/DeBERTa-v3-mnli-fever-anli-v1',   # nli_effect/distilroberta-paraphrase-mnli-fever-anli-v1
#                                                           label2id=label2id, id2label=id2label).to(device)

# free memory
if device == "cuda":
    flush()
    release_memory(model)

datasets_not_to_evaluate = ["none"]  #["banking77", "massive", "manifesto"]   # "anthropic", "banking77", "massive", "manifesto", "empathetic"
nli_datasets = ["mnli_m", "mnli_mm", "fevernli", "anli_r1", "anli_r2", "anli_r3", "wanli", "lingnli"]

result_dic = {}
for key_task_name, value_dataset in tqdm.tqdm(encoded_dataset_test_disaggregated.items(), desc="Iterations over testsets"):
    print(f"\n*** Evaluating task: {key_task_name}. Length of dataset: {len(value_dataset)}")
    with torch.inference_mode():
        # skip selected datasets
        if any(dataset_name in key_task_name for dataset_name in datasets_not_to_evaluate):
            print(f"Dataset skipped: {key_task_name}")
            continue
        # eval not_nli datasets
        elif key_task_name not in nli_datasets:  #dataset_test_disaggregated.keys():
            label_text_alphabetical_task = np.sort(np.unique(value_dataset["label_text"])).tolist()
            trainer.compute_metrics = lambda x: compute_metrics_nli_binary(x, label_text_alphabetical=label_text_alphabetical_task)
            result = trainer.evaluate(eval_dataset=encoded_dataset_test_disaggregated[key_task_name])
        # note: models like bart with 3-class NLI head cannot be evaluated on binary datasets
        elif "bart" in model_name.lower():
            print(f"Dataset skipped: {key_task_name}")
            continue
        elif key_task_name in nli_datasets:
            trainer.compute_metrics = lambda x: compute_metrics_standard(x, label_text_alphabetical=label_text_unique)
            result = trainer.evaluate(eval_dataset=encoded_dataset_test_disaggregated[key_task_name])
        # eval nli datasets. only works for binary nli models because datasets are binary
        #elif len(label_text_unique) == 2:
        #    trainer.compute_metrics = lambda x: compute_metrics_standard(x, label_text_alphabetical=label_text_unique)
        #    result = trainer.evaluate(eval_dataset=encoded_dataset_test_disaggregated[key_task_name])
        else:
            raise ValueError(f"Issue with task: {key_task_name}")

    result_dic.update({key_task_name: result})
    print(f"Result for task {key_task_name}: ", result, "\n")

    # log metrics
    wandb.run.summary.update({f"{key_task_name}/{key_metric_name}": value_metric for key_metric_name, value_metric in result.items()})

    if device == "cuda":
        flush()
        release_memory(model)


print("\n\nOverall results: ", result_dic)


In [ ]:

# log additional info to config
# needs to be done after trainer initialized the config (?)
#wandb.config.update({"dataset_name_heldout": args.dataset_name_heldout}, allow_val_change=True)

print("wandb.run.id: ", wandb.run.id)
print("wandb.run.name: ", wandb.run.name)

wandb.finish()


#### Save

In [ ]:
# save & upload trained models
if args.upload_to_hub and args.do_train:

    trainer.push_to_hub(commit_message="End of training")

    # tokenizer needs to be uploaded separately to create tokenizer.json (use_fast=False)
    # otherwise only tokenizer_config.json is created and pip install sentencepiece is required
    # load use_fast=True tokenizer and upload
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=max_length)  # model_max_length=512
    tokenizer.push_to_hub(repo_id=hub_model_id, use_temp_dir=True, private=True, token=config.HF_ACCESS_TOKEN)


    # to save best model to disk
    """
    model_path = f"{training_directory}/best-{model_name.split('/')[-1]}-{DATE}"

    trainer.save_model(output_dir=model_path)

    print(os.getcwd())
    model = AutoModelForSequenceClassification.from_pretrained(model_path, torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, model_max_length=512)

    ## Push to hub without trainer
    #!sudo apt-get install git-lfs
    #!huggingface-cli login
    # unnecessary if token provided below

    # https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.push_to_hub
    repo_id = f'MoritzLaurer/{model_name.split("/")[-1]}-zeroshot-v1.1-{args.dataset_name_heldout}'
    model.push_to_hub(repo_id=repo_id, use_temp_dir=True, private=True, token=config.HF_ACCESS_TOKEN)
    tokenizer.push_to_hub(repo_id=repo_id, use_temp_dir=True, private=True, token=config.HF_ACCESS_TOKEN)
    """

In [ ]:
## testing automatic creation of .md file
# https://mdutils.readthedocs.io/en/latest/mdutils.html#subpackages
mdFile = MdUtils(file_name=f'README-{model_name.split("/")[-1]}-{DATE}', title='Model Card')

row_dataset_names = list(result_dic.keys())
row_accuracy = [str(round(value["eval_accuracy"], 3)) for key, value in result_dic.items()]
row_f1_macro = [str(round(value["eval_f1_macro"], 3)) for key, value in result_dic.items()]
row_samp_per_sec = [str(round(value["eval_samples_per_second"], 0)) for key, value in result_dic.items()]

print("Warning: These mean calculations are only correct if all NLI datasets are evaluated and if they are evaluated first.")
# mean with nli
row_accuracy_mean = round(np.mean([float(value) for value in row_accuracy]), 3)
row_f1_macro_mean = round(np.mean([float(value) for value in row_f1_macro]), 3)
row_samp_per_sec_mean = round(np.mean([float(value) for value in row_samp_per_sec]), 0)
# mean without 8 nli datasets
row_accuracy_mean_wo_nli = round(np.mean([float(value) for value in row_accuracy[len(nli_datasets):]]), 3)
row_f1_macro_mean_wo_nli = round(np.mean([float(value) for value in row_f1_macro[len(nli_datasets):]]), 3)
row_samp_per_sec_mean_wo_nli = round(np.mean([float(value) for value in row_samp_per_sec][8:]), 0)

table_lst = ["Datasets", "Mean", "Mean w/o NLI"] + row_dataset_names + ["Accuracy", row_accuracy_mean, row_accuracy_mean_wo_nli] + row_accuracy + ["F1 macro", row_f1_macro_mean, row_f1_macro_mean_wo_nli] + row_f1_macro + [f"Inference text/sec (GPU, batch={eval_batch})", row_samp_per_sec_mean, row_samp_per_sec_mean_wo_nli] + row_samp_per_sec

# create markdown table with results
#mdFile.new_line()
results_table_me = mdFile.new_table(columns=len(list(result_dic.keys()))+3, rows=4, text=table_lst, text_align='center')
print(results_table_me)

# write results_table_me to training directors
path_main = os.getcwd()
os.chdir(training_directory)
mdFile.create_md_file()
os.chdir(path_main)
